In [1]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import calendar
import requests
from datetime import datetime

In [2]:
# Incorporate data
df = pd.read_csv('data.csv')

In [3]:
# Initialize the app - incorporate a Dash Bootstrap theme
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

In [4]:
# Create Year Column
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Year'] = df['Timestamp'].dt.year
# Create Day Column
df['Day'] = df['Timestamp'].dt.date
df['Month'] = df['Timestamp'].dt.month

In [5]:
# Proper aggregation:
year_grouped = df.groupby('Year', as_index=False)['Redemption Count'].sum()
day_grouped = df.groupby('Day', as_index=False)['Redemption Count'].sum()
day_grouped_sorted = day_grouped.sort_values(by='Redemption Count', ascending=False)
month_grouped = df.groupby('Month', as_index=False)['Redemption Count'].sum()
month_grouped_sorted = month_grouped.sort_values(by='Redemption Count', ascending=False).reset_index(drop=True)
month_grouped_sorted['Month_Name'] = month_grouped_sorted['Month'].apply(lambda x: calendar.month_name[x])
top_month = month_grouped_sorted.iloc[0,2]
top_month_rd = month_grouped_sorted.iloc[0,1]

In [6]:
# What is the dataset we need in order to drilldown to the hour? 
# Timestamp, Hour-Day-Month-Year, Day-Month-Year, Month-Year, Year, Timestamp_Redemptions, Hour-Day-Month-Year_Redemptions, Day-Month-Year_Redemptions, Month-Year_Redemptions, Year_Redemptions
# Or we can group in plotly?

In [7]:
all_years

NameError: name 'all_years' is not defined

In [8]:
app.layout = dbc.Container([
    dbc.NavbarSimple(brand="Toronto Ferry Dashboard", color="primary", dark=True),
    dbc.Row([
        dbc.Col([
            html.H4("Redemption Counts"),
            dcc.Graph(figure=px.bar(year_grouped, x='Year', y='Redemption Count'))
        ], width=8),
        dbc.Col([
            html.H4("Top 10 Days"),
            dash_table.DataTable(data=day_grouped_sorted.to_dict('records'), page_size=10),
        ], width=4),
        dbc.Col([
            html.H4("Top Month"),
            dbc.Card([
                dbc.CardBody(
                    [
                        html.H3(top_month, className ="card-title"),
                        html.P(int(top_month_rd), className="card-text")
                    ]
                )
            ],
                style={"width": "10rem"},
            )
        ])
    ])
], fluid=True)

# Run the app
if __name__ == '__main__':
    app.run(debug=True)
